# Train Translator

Based on: https://huggingface.co/docs/transformers/tasks/translation

In [1]:
import sys, os, datetime
import json
import torch
import random
import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TranslationPipeline
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

2023-06-06 16:32:20.070054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 16:32:20.182932: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-06 16:32:20.579463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:
2023-06-06 16:32:20.579507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
import cdli
import languages

In [3]:
def get_finetune_model_id(model_id):
    model_dir = f"../results/{model_id}"
    checkpoints = [(os.path.abspath(x), int(os.path.split(x)[1].split("-")[1])) for x in glob.glob(f"{model_dir}/checkpoint-*")]
    checkpoints = sorted(checkpoints, key=lambda x: x[1])[-1]
    return checkpoints[0]

In [4]:
os.environ["WANDB_NOTEBOOK_NAME"] = "TrainTranslatorNew.ipynb"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

source_langs = set(["akk", "sux"])

# target_langs = set(["en", "it", "es", "fr", "de"])
target_langs = set(["en"])

base_model_id = "t5-base"
finetune_model_id = None
# finetune_model_id = get_finetune_model_id("t5-base-p-akksux-en-20220722-173018")

model_max_length = 512
batch_size = 4 if os.path.basename(base_model_id).startswith("t5-large") else (8 if os.path.basename(base_model_id).startswith("t5-base") else 128)

num_train_epochs = 30

is_bi = True
is_finetune = finetune_model_id is not None and len(finetune_model_id) > 1

In [5]:
date_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
flags = ""
suffix = ""
if is_bi:
    flags += "-bi"
if is_finetune:
    flags += "-f"
    suffix += f"-{os.path.basename(os.path.split(finetune_model_id)[0])}-{os.path.basename(finetune_model_id)}"
model_id = f"{os.path.basename(base_model_id)}{flags}-{''.join(sorted(list(source_langs)))}-{''.join(sorted(list(target_langs)))}-{date_id}{suffix}"
model_id

't5-base-bi-akksux-en-20230606-163222'

In [6]:
has_cuda = torch.cuda.is_available()
device = torch.cuda.device(0) if has_cuda else "cpu"
has_cuda, device

(True, <torch.cuda.device at 0x7f0c50d73e80>)

In [7]:
!nvidia-smi

Tue Jun  6 16:32:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 37%   53C    P8    24W / 350W |    351MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Training Data

In [8]:
def get_prefix(src_lang, tgt_lang):
    s = languages.all_languages[src_lang]
    t = languages.all_languages[tgt_lang]
    return f"translate {s} to {t}: "
    
get_prefix("suxts", "es")

'translate Sumerian to Spanish: '

In [9]:
def load_translations_dataset(from_lang, to_lang):
    ds = Dataset.from_json(f"../data/translations_{from_lang}_to_{to_lang}.jsonl")
    srcs = [get_prefix(from_lang, to_lang) + x[from_lang] for x in ds]
    targets = [x[to_lang] for x in ds]
    if is_bi:
        srcs.extend([get_prefix(to_lang, from_lang) + x[to_lang] for x in ds])
        targets.extend([x[from_lang] for x in ds])
#     ds = ds.add_column("source", srcs)
#     ds = ds.add_column("target", targets)
#     ds = ds.remove_columns([from_lang])
#     ds = ds.rename_column(to_lang, "target")
    ds = Dataset.from_dict({"source": srcs, "target": targets})
    return ds
    
translation_datasets = {lang: load_translations_dataset(lang, "en") for lang in source_langs}

Using custom data configuration default-3a61abe3977ee75f
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-3a61abe3977ee75f/0.0.0)
Using custom data configuration default-bf789b25077d4aba
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-bf789b25077d4aba/0.0.0)


In [10]:
translation_datasets

{'akk': Dataset({
     features: ['source', 'target'],
     num_rows: 191258
 }),
 'sux': Dataset({
     features: ['source', 'target'],
     num_rows: 42350
 })}

In [11]:
translation_datasets["sux"][1]

{'source': 'translate Sumerian to English: # (gesz)isimu3(+mu2) - |_gisz_-(U)-_ad-sar_| = %a pe-er-hu-um # (gesz)ildagx(|A-(_gu4_xKUR)|) - |_gisz-a-gu4_xKUR| = %a a-da-ru-um # (gesz)szinig - |_gisz-szinig_|# = %a bi-nu-um # (gesz)asal2 - |_gisz-a-tu-gaba-lisz_|# = %a s,a-ar-ba-tum!(_lum_) # - [...] = %a [...] sza# _dingir#_',
 'target': 'bud poplar tamarisk poplar divine weapon'}

In [12]:
merged_dataset = concatenate_datasets(translation_datasets.values())

In [13]:
merged_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 233608
})

In [14]:
dataset = merged_dataset.shuffle()

In [15]:
dataset[1]

{'source': 'translate Akkadian to English: (d)a-szUr-ma a-na ba-lA-t,i-szu U [sza-lam] [a]-li-szu _bad gal_ sza _uru_-isz-sze sza# pu-zu-ur-(d)a-szUr ru-ba-u a-bi e-pu-szu a-na pi-it-ti _bad_ szu-a-tu _bad_ isz-szé e-pu-usz isz-tu _bad gal_ sza lib-bi-a-lim qa-du _id_ a-na si-hi-ir-ti-[szu] u-la-ab-bi-is-su isz#-tu usz-sze-szu qa#-du sza-ap-ti-szu e-pu-usz',
 'target': 'For his life and the well-being of his city the great wall of New City which Puzur-Ashur III, the prince my forefather, had built — next to that wall I built a new wall. I applied a facing to all of it from the great wall of Inner City as far as the River Tigris. I built it from top to bottom and deposited my clay cone.'}

In [16]:
avg_src_chars_per_token = 1.8338974021110785
avg_tgt_chars_per_token = 2.829482016086902

In [17]:
translations = dataset.train_test_split(test_size=0.1)

In [18]:
train_dataset, test_dataset = translations["train"], translations["test"]

In [19]:
test_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 23361
})

In [20]:
test_dataset[1120:1200]

{'source': ["translate English to Akkadian: He lifts hand-washing water to Anu and sprinkles the king and the people. The king libates from a gold maqqû-vessel to Anu and Antu, and Papsukkal and the king take Anu's hand and lead him from the great dais and he enters his cella and sits down.",
  'translate English to Akkadian: Adallal, adjutant;',
  'translate English to Sumerian: 1 lamb, delivery of Il-DImat, cupbearer,',
  'translate English to Sumerian: roof sheep',
  'translate English to Akkadian: Zabazuna, mighty king, Zili-ewri, ... ...',
  'translate Akkadian to English: ma#-ti-ma ina _egir ud_-_mesz_ ina(!) _szesz_-_mesz dumu_-_mesz im-ri-a im-ri-a_ u sa-lat sza2 _e2_ (disz)_szesz_-_mesz_-sza2-a _dumu_ (disz)hasz-di-ia',
  'translate Akkadian to English: [x x x] :- :- :- _ka_ sa(?)#-a(*)-a(*) 01(*) 1::2(*) _ma_(*)',
  "translate Akkadian to English: ha-'-it,# (d)i2-gi3-gi3",
  'translate Akkadian to English: [u-s,al]-li# a-na _ugu lu_-[x x] [szA _lugal_] _en_-a _kin_(?)-ra# [o]

## Tokenize the Data

In [21]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, model_max_length=model_max_length)

In [22]:
tokenizer.model_max_length

512

In [23]:
print("pad", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos", tokenizer.eos_token, tokenizer.eos_token_id)
print("unk", tokenizer.unk_token, tokenizer.unk_token_id)

pad <pad> 0
eos </s> 1
unk <unk> 2


In [24]:
ccc = 0
sum_src_chars_per_token = 0.0
num_src_chars_per_token = 0
sum_tgt_chars_per_token = 0.0
num_tgt_chars_per_token = 0

def preprocess_function(examples):
    global ccc, sum_src_chars_per_token, sum_tgt_chars_per_token, num_src_chars_per_token, num_tgt_chars_per_token
#     print(examples)
    inputs = [example for example in examples["source"]]
    targets = [example for example in examples["target"]]
    model_inputs = tokenizer(inputs, max_length=model_max_length, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    nexamples = len(inputs)
    for i in range(nexamples):
        nchar = len(inputs[i])
        ntoks = len(model_inputs["input_ids"][i])
        if ntoks > 0:
            sum_src_chars_per_token += nchar / ntoks
            num_src_chars_per_token += 1
        nchar = len(targets[i])
        ntoks = len(model_inputs["labels"][i])
        if ntoks > 0:
            sum_tgt_chars_per_token += nchar / ntoks
            num_tgt_chars_per_token += 1
    
    ccc += 1
    if ccc == 1:
        print(model_inputs["input_ids"][0])
        print(model_inputs["labels"][0])
        nchar = len(targets[0])
        ntoks = len(model_inputs["labels"][0])
        print(nchar, ntoks, nchar / ntoks)

    return model_inputs

tokenized_translations = translations.map(preprocess_function, batched=True)
tokenized_translations

Parameter 'function'=<function preprocess_function at 0x7f0cedfafbe0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/211 [00:00<?, ?ba/s]

[13959, 1566, 12, 12198, 15, 5288, 10, 8003, 3, 7840, 2318, 18, 567, 53, 23, 52, 7, 76, 6, 1]
[220, 599, 2897, 172, 7318, 3, 2168, 7, 702, 115, 519, 3, 40, 76, 7412, 599, 26, 61, 29, 77, 4663, 18, 9427, 357, 4663, 18, 7, 76, 1]
37 29 1.2758620689655173


  0%|          | 0/24 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 210247
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 23361
    })
})

In [25]:
avg_src_chars_per_token = sum_src_chars_per_token / num_src_chars_per_token
avg_tgt_chars_per_token = sum_tgt_chars_per_token / num_tgt_chars_per_token
print("avg_src_chars_per_token", "=", avg_src_chars_per_token)
print("avg_tgt_chars_per_token", "=", avg_tgt_chars_per_token)

avg_src_chars_per_token = 2.554184432226662
avg_tgt_chars_per_token = 2.0533358474041274


In [26]:
tokenized_translations["train"] = tokenized_translations["train"].remove_columns(["source", "target"])
tokenized_translations["test"] = tokenized_translations["test"].remove_columns(["source", "target"])
tokenized_translations

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 210247
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 23361
    })
})

In [27]:
tokenized_translations["train"][0]["labels"][:10]

[220, 599, 2897, 172, 7318, 3, 2168, 7, 702, 115]

## Load the Model

In [28]:
model = AutoModelForSeq2SeqLM.from_pretrained(finetune_model_id if is_finetune else base_model_id, 
                                              max_length=model_max_length)

In [29]:
model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 512,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Ge

## Train

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# data_collator

In [31]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../results/{model_id}",
    evaluation_strategy="epoch",
    learning_rate=2.0e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    fp16=has_cuda,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_translations["train"],
    eval_dataset=tokenized_translations["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



Using amp half precision backend


In [32]:
trainer.train()

/home/fak/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 210247
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 788430
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: praeclarum. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.824300,1.590787
2,1.572300,1.362877
3,1.408500,1.239668
4,1.298700,1.159021
5,1.248300,1.101959
6,1.160000,1.059108
7,1.128900,1.020954
8,1.079400,0.991891
9,1.037300,0.969043
10,1.029100,0.948052


Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-500
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-1000
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-1000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-202

Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-5500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7500
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-7500/special_tokens_map.json
Deleting ol

Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-13500
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-13500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-13500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-13500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-13500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-12000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-14000
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-14000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-14000/pytorch_model.bin
tokenizer config file saved in ../results/t5-b

...

Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-767500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-767500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-767500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-766000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-768000
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-768000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-768000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-768000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-768000/special_tokens_ma

Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-772000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774000
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-772500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774500
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-774500/config.json
Model weights saved i

Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-778500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780500
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-780500/special_tokens_ma

Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-784500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-786500
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-786500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-786500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-786500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-786500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-785000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-787000
Configuration saved in ../results/t5-base-bi-akksux-en-20230606-163222/checkpoint-787000/config.json
Model weights saved i

TrainOutput(global_step=788430, training_loss=1.0353898637384853, metrics={'train_runtime': 121398.3074, 'train_samples_per_second': 51.956, 'train_steps_per_second': 6.495, 'total_flos': 1.4420322591406387e+18, 'train_loss': 1.0353898637384853, 'epoch': 30.0})

## Sample

In [35]:
pipeline = TranslationPipeline(model=model.to("cpu"), tokenizer=tokenizer, max_length=model_max_length)

In [36]:
pipeline

In [37]:
pipeline("translate English to French: hello my name is Frank")

[{'translation_text': 'ah-ra-a : _mu_ (d)frankia'}]

In [38]:
source_test = translations["test"][0]["source"]
target_test = translations["test"][0]["target"]
print(source_test)
print("-"*80)
print(target_test)

translate Sumerian to English: (d)amar-(d)suen (d)en-lil2-le nibru(ki)-a mu pa3-da sag-us2 e2 (d)en-lil2-ka lugal kal-ga lugal uri5(ki)-ma lugal an-ub-da limmu2-ba-ke4 (d)en-ki lugal ki-ag2-ga2-ni-ir abzu ki-ag2-ga2-ni mu-na-du3
--------------------------------------------------------------------------------
Amar-Suena, whiom Enlil in Nippur chose by name, the constant supporter of the temple of Enlil, the mighty king, king fo Ur and king of the four world quarters, for Enki his beloed master, his beloved Apsu-shrine he built for him.


In [39]:
def translate(text):
    return pipeline(text)

translate(source_test)

[{'translation_text': 'For Amar-Suena, whom Enlil in Nippur nominated, the constant supporter of the temple of Enlil, the mighty king, king of Ur and king of the four world quarters, Enki, his beloved king, the Abzu, his beloved temple, he built for him.'}]

## Save to Huggingface

In [42]:
model_path = os.path.abspath(f"/home/fak/nn/Data/generated/cuneiform/{model_id}-fullytrained")
trainer.save_model(model_path)
model_path

Saving model checkpoint to /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained
Configuration saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/config.json
Model weights saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/pytorch_model.bin
tokenizer config file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/tokenizer_config.json
Special tokens file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/special_tokens_map.json


'/home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained'

In [43]:
tokenizer.save_pretrained(model_path)

tokenizer config file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/tokenizer_config.json
Special tokens file saved in /home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/special_tokens_map.json


('/home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/tokenizer_config.json',
 '/home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/special_tokens_map.json',
 '/home/fak/nn/Data/generated/cuneiform/t5-base-bi-akksux-en-20230606-163222-fullytrained/tokenizer.json')